#Homework 2 - Spark

## Lettura dei dati

In [0]:
import requests

from pyspark.sql.functions import when, count, isnull, expr, current_date, col, date_format, year, month, lag, sum, round, max, min, datediff, explode
from pyspark.sql.window import Window

In [0]:
local_path_popolazione_regioni = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/popolazione_regioni.csv"
local_path_dati_covid_regioni = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/dati_covid_regioni.csv"
local_path_dati_vaccini_somministrazioni = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/dati_vaccini_somministrazioni.csv"
local_path_dati_vaccini_consegne = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/dati_vaccini_consegne.csv"

# local_path_popolazione_regioni = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/popolazione_regioni.csv"
# local_path_dati_covid_regioni = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/dati_covid_regioni.csv"
# local_path_dati_vaccini_somministrazioni = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/dati_vaccini_somministrazioni.csv"
# local_path_dati_vaccini_consegne = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/dati_vaccini_consegne.csv"

popolazione_regioni_URL = "https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-statistici-riferimento/popolazione-istat-regione-range.csv"
dati_covid_regioni_URL = "https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv"
dati_vaccini_somministrazioni_URL = "https://raw.githubusercontent.com/italia/covid19-opendata-vaccini/master/dati/somministrazioni-vaccini-latest.csv"
dati_vaccini_consegne_URL = "https://raw.githubusercontent.com/italia/covid19-opendata-vaccini/master/dati/consegne-vaccini-latest.csv"

dbutils.fs.put(local_path_popolazione_regioni, requests.get(popolazione_regioni_URL).text, True)
dbutils.fs.put(local_path_dati_covid_regioni, requests.get(dati_covid_regioni_URL).text, True)
dbutils.fs.put(local_path_dati_vaccini_somministrazioni, requests.get(dati_vaccini_somministrazioni_URL).text, True)
dbutils.fs.put(local_path_dati_vaccini_consegne, requests.get(dati_vaccini_consegne_URL).text, True)

popolazione_regioni = spark.read.option("inferSchema", "true") \
                                .option("header", "true") \
                                .csv(local_path_popolazione_regioni)
dati_covid_regioni = spark.read.option("inferSchema", "true") \
                                .option("header", "true") \
                                .csv(local_path_dati_covid_regioni)
dati_vaccini_somministrazioni = spark.read.option("inferSchema", "true") \
                                          .option("header", "true") \
                                          .csv(local_path_dati_vaccini_somministrazioni)
dati_vaccini_consegne = spark.read.option("inferSchema", "true") \
                                  .option("header", "true") \
                                  .csv(local_path_dati_vaccini_consegne)

Wrote 17300 bytes.
Wrote 1501021 bytes.
Wrote 3965300 bytes.
Wrote 59040 bytes.

## Preprocessing

### Popolazione Regioni
Si eliminano le colonne che non sono di interesse.

In [0]:
popolazione_regioni = popolazione_regioni.drop('codice_regione', 'latitudine_regione', 'longitudine_regione')
popolazione_regioni.show()

+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
codice_nuts_1|descrizione_nuts_1|codice_nuts_2|denominazione_regione|sigla_regione|range_eta|totale_genere_maschile|totale_genere_femminile|totale_generale|
+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
 ITF| Sud| ITF1| Abruzzo| ABR| 0-15| 87423| 82474| 169897|
 ITF| Sud| ITF1| Abruzzo| ABR| 16-19| 24402| 22349| 46751|
 ITF| Sud| ITF1| Abruzzo| ABR| 20-29| 67472| 62611| 130083|
 ITF| Sud| ITF1| Abruzzo| ABR| 30-39| 76472| 74241| 150713|
 ITF| Sud| ITF1| Abruzzo| ABR| 40-49| 94745| 95315| 190060|
 ITF| Sud| ITF1| Abruzzo| ABR| 50-59| 100048| 104109| 204157|
 ITF| Sud| ITF1| Abruzzo| ABR| 60-69| 80545| 87160| 167705|
 ITF| Sud| ITF1| Abruzzo| ABR| 70-79| 60810| 69762| 130572|
 ITF| Sud| ITF1| Abruzzo| ABR| 80-89| 34181| 50307| 84488|
 ITF| Sud| ITF1| Abruzzo| ABR| 90+| 5645| 13870| 19515|
 ITF| Sud| ITF5| Basilicata| BAS| 0-15| 36113| 33881| 69994|
 ITF| Sud| ITF5| Basilicata| BAS| 16-19| 11598| 10434| 22032|
 ITF| Sud| ITF5| Basilicata| BAS| 20-29| 32068| 29115| 61183|
 ITF| Sud| ITF5| Basilicata| BAS| 30-39| 33556| 31648| 65204|
 ITF| Sud| ITF5| Basilicata| BAS| 40-49| 39260| 39207| 78467|
 ITF| Sud| ITF5| Basilicata| BAS| 50-59| 43150| 44672| 87822|
 ITF| Sud| ITF5| Basilicata| BAS| 60-69| 35360| 37457| 72817|
 ITF| Sud| ITF5| Basilicata| BAS| 70-79| 24050| 27755| 51805|
 ITF| Sud| ITF5| Basilicata| BAS| 80-89| 14507| 21600| 36107|
 ITF| Sud| ITF5| Basilicata| BAS| 90+| 2488| 5335| 7823|
+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
only showing top 20 rows

Si verifica che il formato dei dati sia corretto.

In [0]:
popolazione_regioni.dtypes

Out[4]: [('codice_nuts_1', 'string'),
 ('descrizione_nuts_1', 'string'),
 ('codice_nuts_2', 'string'),
 ('denominazione_regione', 'string'),
 ('sigla_regione', 'string'),
 ('range_eta', 'string'),
 ('totale_genere_maschile', 'int'),
 ('totale_genere_femminile', 'int'),
 ('totale_generale', 'int')]

Si verifica la correttezza delle colonne categoriche.

In [0]:
num_regioni = popolazione_regioni.groupBy('denominazione_regione') \
                .count() \
                .count()
print('Il numero di regioni è:', num_regioni)
print()
popolazione_regioni.groupBy('codice_nuts_1', 'descrizione_nuts_1', 'codice_nuts_2', 'denominazione_regione', 'sigla_regione') \
                    .count() \
                    .drop('count') \
                    .orderBy('descrizione_nuts_1') \
                    .show(num_regioni)

Il numero di regioni è: 21

+-------------+------------------+-------------+---------------------+-------------+
codice_nuts_1|descrizione_nuts_1|codice_nuts_2|denominazione_regione|sigla_regione|
+-------------+------------------+-------------+---------------------+-------------+
 ITI| Centro| ITI4| Lazio| LAZ|
 ITI| Centro| ITI3| Marche| MAR|
 ITI| Centro| ITI2| Umbria| UMB|
 ITI| Centro| ITI1| Toscana| TOS|
 ITG| Isole| ITG1| Sicilia| SIC|
 ITG| Isole| ITG2| Sardegna| SAR|
 ITH| Nord-Est| ITH5| Emilia-Romagna| EMR|
 ITH| Nord-Est| ITH1| Bolzano| BOL|
 ITH| Nord-Est| ITH4| Friuli Venezia Gi...| FVG|
 ITH| Nord-Est| ITH2| Trento| TRE|
 ITH| Nord-Est| ITH3| Veneto| VEN|
 ITC| Nord-Ovest| ITC1| Piemonte| PIE|
 ITC| Nord-Ovest| ITC2| Valle d'Aosta| VDA|
 ITC| Nord-Ovest| ITC3| Liguria| LIG|
 ITC| Nord-Ovest| ITC4| Lombardia| LOM|
 ITF| Sud| ITF1| Abruzzo| ABR|
 ITF| Sud| ITF2| Molise| MOL|
 ITF| Sud| ITF3| Campania| CAM|
 ITF| Sud| ITF5| Basilicata| BAS|
 ITF| Sud| ITF4| Puglia| PUG|
 ITF| Sud| ITF6| Calabria| CAL|
+-------------+------------------+-------------+---------------------+-------------+

In [0]:
popolazione_regioni.groupBy('range_eta') \
                    .count() \
                    .orderBy('range_eta') \
                    .show()

+---------+-----+
range_eta|count|
+---------+-----+
 0-15| 21|
 16-19| 21|
 20-29| 21|
 30-39| 21|
 40-49| 21|
 50-59| 21|
 60-69| 21|
 70-79| 21|
 80-89| 21|
 90+| 21|
+---------+-----+

Si verifica la correttezza delle colonne numeriche.

In [0]:
popolazione_regioni.describe(['totale_genere_maschile', 'totale_genere_femminile', 'totale_generale']) \
                    .show()

+-------+----------------------+-----------------------+-----------------+
summary|totale_genere_maschile|totale_genere_femminile| totale_generale|
+-------+----------------------+-----------------------+-----------------+
 count| 210| 210| 210|
 mean| 138333.79047619048| 145673.29523809525|284007.0857142857|
 stddev| 151728.9786894938| 152578.20019731886|303588.4857882117|
 min| 457| 1307| 1764|
 max| 795710| 796399| 1592109|
+-------+----------------------+-----------------------+-----------------+

Si verifica la presenza di *missing values*.

In [0]:
popolazione_regioni.select([count(when(isnull(c), c)).alias(c) for c in popolazione_regioni.columns]).show()

+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
codice_nuts_1|descrizione_nuts_1|codice_nuts_2|denominazione_regione|sigla_regione|range_eta|totale_genere_maschile|totale_genere_femminile|totale_generale|
+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
 0| 0| 0| 0| 0| 0| 0| 0| 0|
+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+

### Dati Covid Regioni
Si eliminano le colonne che non sono di interesse.

In [0]:
dati_covid_regioni = dati_covid_regioni.drop('stato', 'codice_regione', 'lat', 'long', 'note_test', 'note_casi', 'note',
                                             'casi_da_sospetto_diagnostico', 'casi_da_screening',
                                             'casi_testati', 'ingressi_terapia_intensiva', 
                                             'totale_positivi_test_molecolare', 'totale_positivi_test_antigenico_rapido',
                                             'tamponi_test_molecolare', 'tamponi_test_antigenico_rapido')
dati_covid_regioni.show()

+-------------------+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
 data|denominazione_regione|ricoverati_con_sintomi|terapia_intensiva|totale_ospedalizzati|isolamento_domiciliare|totale_positivi|variazione_totale_positivi|nuovi_positivi|dimessi_guariti|deceduti|totale_casi|tamponi|codice_nuts_1|codice_nuts_2|
+-------------------+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
2020-02-24 18:00:00| Abruzzo| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 5| null| null|
2020-02-24 18:00:00| Basilicata| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| null| null|
2020-02-24 18:00:00| Calabria| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| null| null|
2020-02-24 18:00:00| Campania| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 10| null| null|
2020-02-24 18:00:00| Emilia-Romagna| 10| 2| 12| 6| 18| 0| 18| 0| 0| 18| 148| null| null|
2020-02-24 18:00:00| Friuli Venezia Gi...| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 58| null| null|
2020-02-24 18:00:00| Lazio| 1| 1| 2| 0| 2| 0| 2| 1| 0| 3| 124| null| null|
2020-02-24 18:00:00| Liguria| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| null| null|
2020-02-24 18:00:00| Lombardia| 76| 19| 95| 71| 166| 0| 166| 0| 6| 172| 1463| null| null|
2020-02-24 18:00:00| Marche| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 16| null| null|
2020-02-24 18:00:00| Molise| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| null| null|
2020-02-24 18:00:00| P.A. Bolzano| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| null| null|
2020-02-24 18:00:00| P.A. Trento| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 3| null| null|
2020-02-24 18:00:00| Piemonte| 2| 0| 2| 1| 3| 0| 3| 0| 0| 3| 141| null| null|
2020-02-24 18:00:00| Puglia| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| null| null|
2020-02-24 18:00:00| Sardegna| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| null| null|
2020-02-24 18:00:00| Sicilia| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 5| null| null|
2020-02-24 18:00:00| Toscana| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 140| null| null|
2020-02-24 18:00:00| Umbria| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| null| null|
2020-02-24 18:00:00| Valle d'Aosta| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 7| null| null|
+-------------------+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
only showing top 20 rows

Si verifica che il formato dei dati sia corretto.

In [0]:
dati_covid_regioni.dtypes

Out[10]: [('data', 'timestamp'),
 ('denominazione_regione', 'string'),
 ('ricoverati_con_sintomi', 'int'),
 ('terapia_intensiva', 'int'),
 ('totale_ospedalizzati', 'int'),
 ('isolamento_domiciliare', 'int'),
 ('totale_positivi', 'int'),
 ('variazione_totale_positivi', 'int'),
 ('nuovi_positivi', 'int'),
 ('dimessi_guariti', 'int'),
 ('deceduti', 'int'),
 ('totale_casi', 'int'),
 ('tamponi', 'int'),
 ('codice_nuts_1', 'string'),
 ('codice_nuts_2', 'string')]

Si verifica la correttezza delle colonne categoriche.

In [0]:
num_regioni = dati_covid_regioni.groupBy('denominazione_regione') \
                .count() \
                .count()
print('Il numero di regioni è:', num_regioni)
print()
dati_covid_regioni.groupBy('codice_nuts_1', 'codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .orderBy('codice_nuts_1') \
                    .show(50)

Il numero di regioni è: 21

+-------------+-------------+---------------------+
codice_nuts_1|codice_nuts_2|denominazione_regione|
+-------------+-------------+---------------------+
 null| null| Lazio|
 null| null| Sicilia|
 null| null| Emilia-Romagna|
 null| null| Toscana|
 null| null| Puglia|
 null| null| Abruzzo|
 null| null| Sardegna|
 null| null| Umbria|
 null| null| Piemonte|
 null| null| P.A. Bolzano|
 null| null| Molise|
 null| null| Calabria|
 null| null| Friuli Venezia Gi...|
 null| null| Valle d'Aosta|
 null| null| P.A. Trento|
 null| null| Marche|
 null| null| Basilicata|
 null| null| Veneto|
 null| null| Campania|
 null| null| Lombardia|
 null| null| Liguria|
 1| null| P.A. Bolzano|
 172410| ITI| Umbria|
 500340| ITH| P.A. Bolzano|
 ITC| ITC2| Valle d'Aosta|
 ITC| ITC3| Liguria|
 ITC| ITC1| Piemonte|
 ITC| ITC4| Lombardia|
 ITF| ITF6| Calabria|
 ITF| ITF2| Molise|
 ITF| ITF5| Basilicata|
 ITF| ITF1| Abruzzo|
 ITF| ITF3| Campania|
 ITF| ITF4| Puglia|
 ITG| ITG1| Sicilia|
 ITG| ITG2| Sardegna|
 ITH| ITH3| Veneto|
 ITH| ITH5| Emilia-Romagna|
 ITH| ITH2| P.A. Trento|
 ITH| ITH1| P.A. Bolzano|
 ITH| ITH4| Friuli Venezia Gi...|
 ITI| ITI1| Toscana|
 ITI| ITI2| Umbria|
 ITI| ITI3| Marche|
 ITI| ITI4| Lazio|
+-------------+-------------+---------------------+

Si possono fare alcune osservazioni:
- per alcune entry mancano i codici nuts;
- per aclune entry i codici nuts sono sbagliati (problema dovuto alla lettura del file csv);
- alcune regioni (Bolzano e Trento) hanno una denominazione diversa rispetto alla tabella `popolazione_regioni`;

In [0]:
join1 = popolazione_regioni.join(dati_covid_regioni, on='denominazione_regione', how='anti') \
                    .groupBy('codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .withColumnRenamed('denominazione_regione', 'popolazione_regioni_denominazione_regione')

join2 = dati_covid_regioni.join(popolazione_regioni, on='denominazione_regione', how='anti') \
                    .groupBy('codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .withColumnRenamed('denominazione_regione', 'dati_covid_regioni_denominazione_regione')

join1.join(join2, on='codice_nuts_2').show()

dati_covid_regioni = dati_covid_regioni.replace('P.A. Trento', value='Trento', subset='denominazione_regione')
dati_covid_regioni = dati_covid_regioni.replace('P.A. Bolzano', value='Bolzano', subset='denominazione_regione')

+-------------+-----------------------------------------+----------------------------------------+
codice_nuts_2|popolazione_regioni_denominazione_regione|dati_covid_regioni_denominazione_regione|
+-------------+-----------------------------------------+----------------------------------------+
 ITH1| Bolzano| P.A. Bolzano|
 ITH2| Trento| P.A. Trento|
+-------------+-----------------------------------------+----------------------------------------+

In [0]:
dati_covid_regioni = dati_covid_regioni.drop('codice_nuts_1', 'codice_nuts_2')

data = popolazione_regioni.groupBy('codice_nuts_1', 'codice_nuts_2', 'denominazione_regione') \
                          .count() \
                          .drop('count')

dati_covid_regioni = dati_covid_regioni.join(data, on=['denominazione_regione'], how='left')

dati_covid_regioni.groupBy('codice_nuts_1', 'codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .orderBy('codice_nuts_1').show(num_regioni)

+-------------+-------------+---------------------+
codice_nuts_1|codice_nuts_2|denominazione_regione|
+-------------+-------------+---------------------+
 ITC| ITC2| Valle d'Aosta|
 ITC| ITC3| Liguria|
 ITC| ITC1| Piemonte|
 ITC| ITC4| Lombardia|
 ITF| ITF4| Puglia|
 ITF| ITF1| Abruzzo|
 ITF| ITF6| Calabria|
 ITF| ITF2| Molise|
 ITF| ITF5| Basilicata|
 ITF| ITF3| Campania|
 ITG| ITG2| Sardegna|
 ITG| ITG1| Sicilia|
 ITH| ITH3| Veneto|
 ITH| ITH5| Emilia-Romagna|
 ITH| ITH1| Bolzano|
 ITH| ITH2| Trento|
 ITH| ITH4| Friuli Venezia Gi...|
 ITI| ITI1| Toscana|
 ITI| ITI2| Umbria|
 ITI| ITI3| Marche|
 ITI| ITI4| Lazio|
+-------------+-------------+---------------------+

Si verifica la correttezza delle colonne numeriche.

In [0]:
dati_covid_regioni.describe(['ricoverati_con_sintomi', 'terapia_intensiva', 'totale_ospedalizzati', 'dimessi_guariti']) \
                  .show()

+-------+----------------------+------------------+--------------------+------------------+
summary|ricoverati_con_sintomi| terapia_intensiva|totale_ospedalizzati| dimessi_guariti|
+-------+----------------------+------------------+--------------------+------------------+
 count| 9051| 9051| 9051| 9051|
 mean| 712.0317092034029| 81.62269362501381| 793.6544028284168|41720.882664898905|
 stddev| 1290.6646574108404|143.02273531693888| 1428.9305275989084| 86416.42512081977|
 min| 0| 0| 0| 0|
 max| 12077| 1381| 13328| 717503|
+-------+----------------------+------------------+--------------------+------------------+

In [0]:
dati_covid_regioni.describe(['isolamento_domiciliare', 'totale_positivi', 'variazione_totale_positivi', 'nuovi_positivi', 'deceduti', 'totale_casi', 'tamponi']) \
                  .show()

+-------+----------------------+------------------+--------------------------+-----------------+-----------------+------------------+-----------------+
summary|isolamento_domiciliare| totale_positivi|variazione_totale_positivi| nuovi_positivi| deceduti| totale_casi| tamponi|
+-------+----------------------+------------------+--------------------------+-----------------+-----------------+------------------+-----------------+
 count| 9051| 9051| 9051| 9051| 9051| 9051| 9051|
 mean| 11655.920892719036|12449.575295547453| 48.44635951828527|442.2650535852392|2462.577505248039| 56633.03745442493|844168.3337752734|
 stddev| 21230.129780605814|22232.931712089026| 826.3479228393714|854.3477761172638|4685.474952845612|105988.09255231376|1331181.422682871|
 min| 0| 0| -21926| -229| 0| 0| 0|
 max| 155066| 164406| 10263| 11489| 32829| 802406| 9421651|
+-------+----------------------+------------------+--------------------------+-----------------+-----------------+------------------+-----------------+

In [0]:
dati_covid_regioni.filter(condition='nuovi_positivi < 0').show()

+---------------------+-------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
denominazione_regione| data|ricoverati_con_sintomi|terapia_intensiva|totale_ospedalizzati|isolamento_domiciliare|totale_positivi|variazione_totale_positivi|nuovi_positivi|dimessi_guariti|deceduti|totale_casi|tamponi|codice_nuts_1|codice_nuts_2|
+---------------------+-------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
 Piemonte|2020-02-27 18:00:00| 2| 0| 2| 0| 2| -1| -1| 0| 0| 2| 156| ITC| ITC1|
 Liguria|2020-03-01 17:00:00| 12| 1| 13| 8| 21| -17| -17| 4| 0| 25| 121| ITC| ITC3|
 Liguria|2020-03-02 18:00:00| 12| 1| 13| 5| 18| -3| -3| 4| 0| 22| 121| ITC| ITC3|
 Sicilia|2020-03-02 18:00:00| 2| 0| 2| 3| 5| -2| -2| 2| 0| 7| 307| ITG| ITG1|
 Piemonte|2020-03-09 18:00:00| 222| 50| 272| 65| 337| -18| -10| 0| 13| 350| 1681| ITC| ITC1|
 Calabria|2020-04-17 17:00:00| 154| 7| 161| 658| 819| -28| -18| 99| 73| 991| 21657| ITF| ITF6|
 Sardegna|2020-05-04 17:00:00| 91| 9| 100| 553| 653| -36| -2| 545| 119| 1317| 28052| ITG| ITG2|
 Basilicata|2020-05-07 17:00:00| 48| 2| 50| 105| 155| -17| -16| 202| 26| 383| 16272| ITF| ITF5|
 Basilicata|2020-05-08 17:00:00| 48| 2| 50| 102| 152| -3| -1| 204| 26| 382| 16777| ITF| ITF5|
 Marche|2020-05-19 17:00:00| 144| 17| 161| 1967| 2128| -187| -3| 3561| 986| 6675| 89985| ITI| ITI3|
 Sardegna|2020-05-25 17:00:00| 49| 3| 52| 179| 231| -14| -2| 994| 129| 1354| 51073| ITG| ITG2|
 Sardegna|2020-06-09 17:00:00| 9| 1| 10| 44| 54| -2| -1| 1176| 131| 1361| 64272| ITG| ITG2|
 Campania|2020-06-12 17:00:00| 61| 2| 63| 283| 346| -264| -229| 3832| 430| 4608| 230551| ITF| ITF3|
+---------------------+-------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+

Si verifica la presenza di *missing values*.

In [0]:
dati_covid_regioni.select([count(when(isnull(c), c)).alias(c) for c in dati_covid_regioni.drop('data').columns]).show()

+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
denominazione_regione|ricoverati_con_sintomi|terapia_intensiva|totale_ospedalizzati|isolamento_domiciliare|totale_positivi|variazione_totale_positivi|nuovi_positivi|dimessi_guariti|deceduti|totale_casi|tamponi|codice_nuts_1|codice_nuts_2|
+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+

Si verifica la presenza di *missing values* nella serie temporale.

In [0]:
min_data = dati_covid_regioni.agg(min('data').alias('minimo'))
max_data = dati_covid_regioni.agg(max('data').alias('massimo'))

date = min_data.join(max_data)
date = date.withColumn('minimo', date_format(date.minimo, 'yyyy-MM-dd').cast('date')) \
           .withColumn('massimo', date_format(date.massimo, 'yyyy-MM-dd').cast('date'))

date.show()

+----------+----------+
 minimo| massimo|
+----------+----------+
2020-02-24|2021-04-29|
+----------+----------+

In [0]:
time_series = date.withColumn('data', explode(expr('sequence(minimo, massimo, interval 1 day)'))) \
                  .select('data')

time_series.show()

+----------+
 data|
+----------+
2020-02-24|
2020-02-25|
2020-02-26|
2020-02-27|
2020-02-28|
2020-02-29|
2020-03-01|
2020-03-02|
2020-03-03|
2020-03-04|
2020-03-05|
2020-03-06|
2020-03-07|
2020-03-08|
2020-03-09|
2020-03-10|
2020-03-11|
2020-03-12|
2020-03-13|
2020-03-14|
+----------+
only showing top 20 rows

In [0]:
date_covid = dati_covid_regioni.select('data') \
                               .withColumn('data', date_format(dati_covid_regioni.data, 'yyyy-MM-dd').cast('date')) \
                               .distinct() \
                               .orderBy('data')

date_covid.join(time_series, on='data', how='anti').show()

+----+
data|
+----+
+----+

### Dati Somministrazioni Vaccini

Si elmininano le colonne che non sono di interesse

In [0]:
dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.drop('area', 'codice_regione_ISTAT')
dati_vaccini_somministrazioni.show()

+---------------------+---------------+-----------------+--------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+----------+
data_somministrazione| fornitore|fascia_anagrafica|sesso_maschile|sesso_femminile|categoria_operatori_sanitari_sociosanitari|categoria_personale_non_sanitario|categoria_ospiti_rsa|categoria_60_69|categoria_70_79|categoria_over80|categoria_forze_armate|categoria_personale_scolastico|categoria_soggetti_fragili|categoria_altro|prima_dose|seconda_dose|codice_NUTS1|codice_NUTS2| nome_area|
+---------------------+---------------+-----------------+--------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+----------+
 2020-12-27|Pfizer/BioNTech| 20-29| 1| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 30-39| 1| 3| 4| 0| 0| 0| 0| 0| 0| 0| 0| 0| 4| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 40-49| 1| 6| 7| 0| 0| 0| 0| 0| 0| 0| 0| 0| 7| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 50-59| 4| 5| 9| 0| 0| 0| 0| 0| 0| 0| 0| 0| 9| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 60-69| 10| 4| 13| 1| 0| 0| 0| 0| 0| 0| 0| 0| 14| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 70-79| 1| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 1| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 80-89| 1| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 1| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 20-29| 4| 5| 9| 0| 0| 0| 0| 0| 0| 0| 0| 0| 9| 0| ITF| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 30-39| 10| 18| 27| 0| 0| 0| 0| 0| 0| 0| 0| 1| 28| 0| ITF| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 40-49| 7| 23| 29| 0| 0| 0| 0| 0| 0| 0| 0| 1| 30| 0| ITF| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 50-59| 11| 15| 26| 0| 0| 0| 0| 0| 0| 0| 0| 0| 26| 0| ITF| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 60-69| 9| 4| 13| 0| 0| 0| 0| 0| 0| 0| 0| 0| 13| 0| ITF| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 20-29| 15| 34| 49| 0| 0| 0| 0| 0| 0| 0| 0| 0| 49| 0| ITF| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 30-39| 23| 20| 43| 0| 0| 0| 0| 0| 0| 0| 0| 0| 43| 0| ITF| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 40-49| 23| 27| 50| 0| 0| 0| 0| 0| 0| 0| 0| 0| 50| 0| ITF| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 50-59| 49| 28| 76| 1| 0| 0| 0| 0| 0| 0| 0| 0| 77| 0| ITF| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 60-69| 44| 16| 60| 0| 0| 0| 0| 0| 0| 0| 0| 0| 60| 0| ITF| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 20-29| 27| 33| 59| 1| 0| 0| 0| 0| 0| 0| 0| 0| 60| 0| ITF| ITF3| Campania|
 2020-12-27|Pfizer/BioNTech| 30-39| 61| 64| 122| 2| 0| 0| 0| 0| 0| 0| 0| 1| 125| 0| ITF| ITF3| Campania|
 2020-12-27|Pfizer/BioNTech| 40-49| 80| 87| 164| 3| 0| 0| 0| 0| 0| 0| 0| 0| 167| 0| ITF| ITF3| Campania|
+---------------------+---------------+-----------------+--------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+----------+
only showing top 20 rows

Si verifica che il formato dei dati sia corretto

In [0]:
dati_vaccini_somministrazioni.dtypes

Out[22]: [('data_somministrazione', 'string'),
 ('fornitore', 'string'),
 ('fascia_anagrafica', 'string'),
 ('sesso_maschile', 'int'),
 ('sesso_femminile', 'int'),
 ('categoria_operatori_sanitari_sociosanitari', 'int'),
 ('categoria_personale_non_sanitario', 'int'),
 ('categoria_ospiti_rsa', 'int'),
 ('categoria_60_69', 'int'),
 ('categoria_70_79', 'int'),
 ('categoria_over80', 'int'),
 ('categoria_forze_armate', 'int'),
 ('categoria_personale_scolastico', 'int'),
 ('categoria_soggetti_fragili', 'int'),
 ('categoria_altro', 'int'),
 ('prima_dose', 'int'),
 ('seconda_dose', 'int'),
 ('codice_NUTS1', 'string'),
 ('codice_NUTS2', 'string'),
 ('nome_area', 'string')]

Si verifica la correttezza delle colonne categoriche.

In [0]:
num_regioni = dati_vaccini_somministrazioni.groupBy('nome_area') \
                                           .count() \
                                           .count()
print('Il numero di regioni è:', num_regioni)
print()
dati_vaccini_somministrazioni.groupBy('codice_NUTS1', 'codice_NUTS2', 'nome_area') \
                             .count() \
                             .drop('count') \
                             .orderBy('codice_NUTS1') \
                             .show(50)

Il numero di regioni è: 21

+------------+------------+--------------------+
codice_NUTS1|codice_NUTS2| nome_area|
+------------+------------+--------------------+
 ITC| ITC2|Valle d'Aosta / V...|
 ITC| ITC3| Liguria|
 ITC| ITC1| Piemonte|
 ITC| ITC4| Lombardia|
 ITF| ITF4| Puglia|
 ITF| ITF1| Abruzzo|
 ITF| ITF6| Calabria|
 ITF| ITF2| Molise|
 ITF| ITF5| Basilicata|
 ITF| ITF3| Campania|
 ITG| ITG2| Sardegna|
 ITG| ITG1| Sicilia|
 ITH| ITH5| Emilia-Romagna|
 ITH| ITH1|Provincia Autonom...|
 ITH| ITH4|Friuli-Venezia Gi...|
 ITH| ITH3| Veneto|
 ITH| ITH2|Provincia Autonom...|
 ITI| ITI1| Toscana|
 ITI| ITI2| Umbria|
 ITI| ITI3| Marche|
 ITI| ITI4| Lazio|
+------------+------------+--------------------+

In [0]:
join1 = popolazione_regioni.join(dati_vaccini_somministrazioni, popolazione_regioni.denominazione_regione == dati_vaccini_somministrazioni.nome_area, how='anti') \
                    .groupBy('codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') 

join2 = dati_vaccini_somministrazioni.join(popolazione_regioni, popolazione_regioni.denominazione_regione == dati_vaccini_somministrazioni.nome_area, how='anti') \
                    .groupBy('codice_NUTS2', 'nome_area') \
                    .count() \
                    .drop('count') 

join1.join(join2, popolazione_regioni.codice_nuts_2 == dati_vaccini_somministrazioni.codice_NUTS2).show()

dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.replace('Provincia Autonoma Trento', value='Trento', subset='nome_area')
dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.replace('Provincia Autonoma Bolzano / Bozen', value='Bolzano', subset='nome_area')
dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.replace('Friuli-Venezia Giulia', value='Friuli Venezia Giulia', subset='nome_area')
dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.replace('Valle d\'Aosta / Vallée d\'Aoste', value='Valle d\'Aosta', subset='nome_area')

+-------------+---------------------+------------+--------------------+
codice_nuts_2|denominazione_regione|codice_NUTS2| nome_area|
+-------------+---------------------+------------+--------------------+
 ITC2| Valle d'Aosta| ITC2|Valle d'Aosta / V...|
 ITH2| Trento| ITH2|Provincia Autonom...|
 ITH4| Friuli Venezia Gi...| ITH4|Friuli-Venezia Gi...|
 ITH1| Bolzano| ITH1|Provincia Autonom...|
+-------------+---------------------+------------+--------------------+

In [0]:
dati_vaccini_somministrazioni.groupBy('codice_NUTS1', 'codice_NUTS2', 'nome_area') \
                             .count() \
                             .drop('count') \
                             .orderBy('codice_NUTS1') \
                             .show(num_regioni)

+------------+------------+--------------------+
codice_NUTS1|codice_NUTS2| nome_area|
+------------+------------+--------------------+
 ITC| ITC2| Valle d'Aosta|
 ITC| ITC3| Liguria|
 ITC| ITC1| Piemonte|
 ITC| ITC4| Lombardia|
 ITF| ITF4| Puglia|
 ITF| ITF1| Abruzzo|
 ITF| ITF6| Calabria|
 ITF| ITF2| Molise|
 ITF| ITF5| Basilicata|
 ITF| ITF3| Campania|
 ITG| ITG2| Sardegna|
 ITG| ITG1| Sicilia|
 ITH| ITH3| Veneto|
 ITH| ITH5| Emilia-Romagna|
 ITH| ITH1| Bolzano|
 ITH| ITH2| Trento|
 ITH| ITH4|Friuli Venezia Gi...|
 ITI| ITI1| Toscana|
 ITI| ITI2| Umbria|
 ITI| ITI3| Marche|
 ITI| ITI4| Lazio|
+------------+------------+--------------------+

In [0]:
dati_vaccini_somministrazioni.groupBy('fascia_anagrafica') \
                    .count() \
                    .drop('count') \
                    .orderBy('fascia_anagrafica') \
                    .show()

+-----------------+
fascia_anagrafica|
+-----------------+
 16-19|
 20-29|
 30-39|
 40-49|
 50-59|
 60-69|
 70-79|
 80-89|
 90+|
+-----------------+

Si osserva che rispetto alla tabella popolazione_regioni manca la `fascia_anagrafica` 0-15

Si verifica la correttezza delle colonne numeriche.

In [0]:
dati_vaccini_somministrazioni.describe(['sesso_maschile', 'sesso_femminile', 'prima_dose', 'seconda_dose']) \
                             .show()

+-------+------------------+------------------+-----------------+------------------+
summary| sesso_maschile| sesso_femminile| prima_dose| seconda_dose|
+-------+------------------+------------------+-----------------+------------------+
 count| 42192| 42192| 42192| 42192|
 mean|197.13914960182026|263.10485400075845|324.8812571103527|135.36274649222602|
 stddev| 562.1440684873028| 713.8935120725424|985.5663166890031| 545.3299642921239|
 min| 0| 0| 0| 0|
 max| 13949| 18438| 29494| 18545|
+-------+------------------+------------------+-----------------+------------------+

In [0]:
dati_vaccini_somministrazioni.describe(['categoria_operatori_sanitari_sociosanitari', 'categoria_personale_non_sanitario', 'categoria_ospiti_rsa', 'categoria_60_69', 'categoria_70_79', 'categoria_over80', 'categoria_forze_armate', 'categoria_personale_scolastico', 'categoria_soggetti_fragili', 'categoria_altro']) \
                             .show()

+-------+------------------------------------------+---------------------------------+--------------------+------------------+-----------------+------------------+----------------------+------------------------------+--------------------------+-----------------+
summary|categoria_operatori_sanitari_sociosanitari|categoria_personale_non_sanitario|categoria_ospiti_rsa| categoria_60_69| categoria_70_79| categoria_over80|categoria_forze_armate|categoria_personale_scolastico|categoria_soggetti_fragili| categoria_altro|
+-------+------------------------------------------+---------------------------------+--------------------+------------------+-----------------+------------------+----------------------+------------------------------+--------------------------+-----------------+
 count| 42192| 42192| 42192| 42192| 42192| 42192| 42192| 42192| 42192| 42192|
 mean| 76.48703545695867| 20.99101725445582| 15.439087978763746|23.120544178991278|67.81899412210845|143.93188282138794| 7.63241846795601| 27.468406332954114| 68.9802332195677|8.374383769434964|
 stddev| 228.21436631476394| 66.53513519907604| 61.45778248632519|298.78870933053304|670.9219174066919| 908.0349118865371| 42.83052634857925| 152.57224335673274| 297.65807073296054|38.52614321568288|
 min| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
 max| 4661| 2303| 2316| 20021| 28028| 31047| 1214| 3583| 7386| 2008|
+-------+------------------------------------------+---------------------------------+--------------------+------------------+-----------------+------------------+----------------------+------------------------------+--------------------------+-----------------+

Si verifica la presenza di *missing values*.

In [0]:
dati_vaccini_somministrazioni.select([count(when(isnull(c), c)).alias(c) for c in dati_vaccini_somministrazioni.drop('data').columns]).show()

+---------------------+---------+-----------------+--------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+---------+
data_somministrazione|fornitore|fascia_anagrafica|sesso_maschile|sesso_femminile|categoria_operatori_sanitari_sociosanitari|categoria_personale_non_sanitario|categoria_ospiti_rsa|categoria_60_69|categoria_70_79|categoria_over80|categoria_forze_armate|categoria_personale_scolastico|categoria_soggetti_fragili|categoria_altro|prima_dose|seconda_dose|codice_NUTS1|codice_NUTS2|nome_area|
+---------------------+---------+-----------------+--------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+---------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+---------------------+---------+-----------------+--------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+---------+

Si verifica la presenza di *missing values* nella serie temporale.

In [0]:
min_data = dati_vaccini_somministrazioni.agg(min('data_somministrazione').alias('minimo'))
max_data = dati_vaccini_somministrazioni.agg(max('data_somministrazione').alias('massimo'))

date = min_data.join(max_data)
date = date.withColumn('minimo', date_format(date.minimo, 'yyyy-MM-dd').cast('date')) \
           .withColumn('massimo', date_format(date.massimo, 'yyyy-MM-dd').cast('date'))

date.show()

+----------+----------+
 minimo| massimo|
+----------+----------+
2020-12-27|2021-04-29|
+----------+----------+

In [0]:
time_series = date.withColumn('data_somministrazione', explode(expr('sequence(minimo, massimo, interval 1 day)'))) \
                  .select('data_somministrazione')

time_series.show()

+---------------------+
data_somministrazione|
+---------------------+
 2020-12-27|
 2020-12-28|
 2020-12-29|
 2020-12-30|
 2020-12-31|
 2021-01-01|
 2021-01-02|
 2021-01-03|
 2021-01-04|
 2021-01-05|
 2021-01-06|
 2021-01-07|
 2021-01-08|
 2021-01-09|
 2021-01-10|
 2021-01-11|
 2021-01-12|
 2021-01-13|
 2021-01-14|
 2021-01-15|
+---------------------+
only showing top 20 rows

In [0]:
date_covid = dati_vaccini_somministrazioni.select('data_somministrazione') \
                               .withColumn('data_somministrazione', date_format(dati_vaccini_somministrazioni.data_somministrazione, 'yyyy-MM-dd').cast('date')) \
                               .distinct() \
                               .orderBy('data_somministrazione')

date_covid.join(time_series, on='data_somministrazione', how='anti') \
          .show()

+---------------------+
data_somministrazione|
+---------------------+
+---------------------+

## Analisi

### Dati covid Regioni

#### Panoramica Casi per Mese

In [0]:
casi_mensili = dati_covid_regioni.select(date_format('data','yyyy-MM').alias('mese'), 'nuovi_positivi') \
                  .groupby('mese') \
                  .sum('nuovi_positivi') \
                  .withColumnRenamed('sum(nuovi_positivi)', 'casi')

In [0]:
tamponi_mensili = dati_covid_regioni.select(date_format('data','yyyy-MM').alias('mese'), 'denominazione_regione', 'tamponi') \
                                    .groupby('mese', 'denominazione_regione') \
                                    .max('tamponi') \
                                    .groupby('mese') \
                                    .sum('max(tamponi)') \
                                    .withColumnRenamed('sum(max(tamponi))', 'tamponi')

my_window = Window.partitionBy().orderBy("mese") 

tamponi_mensili = tamponi_mensili.withColumn("prev_value", lag(tamponi_mensili.tamponi) \
                                 .over(my_window)) 
tamponi_mensili = tamponi_mensili.withColumn("diff", when(isnull(tamponi_mensili.tamponi - tamponi_mensili.prev_value), tamponi_mensili.tamponi).otherwise(tamponi_mensili.tamponi - tamponi_mensili.prev_value)) \
                                 .drop('tamponi', 'prev_value') \
                                 .withColumnRenamed('diff', 'tamponi')

In [0]:
panoramica_mensile = casi_mensili.join(tamponi_mensili, on='mese') \
                                 .orderBy('mese', ascending=False)

panoramica_mensile = panoramica_mensile.withColumn('percentuale_tamponi_positivi', panoramica_mensile.casi / panoramica_mensile.tamponi * 100)

panoramica_mensile.show()

+-------+------+-------+----------------------------+
 mese| casi|tamponi|percentuale_tamponi_positivi|
+-------+------+-------+----------------------------+
2021-04|426000|8338484| 5.108842326734692|
2021-03|648200|9769610| 6.634860552263601|
2021-02|372503|7466161| 4.989217350121435|
2021-01|445585|5948709| 7.4904487679595695|
2020-12|508914|4772918| 10.662533904835575|
2020-11|922206|6160638| 14.969326228874348|
2020-10|364607|4450539| 8.192423434554781|
2020-09| 45623|2689063| 1.6966132812805055|
2020-08| 21702|1831746| 1.1847712510358968|
2020-07| 7006|1423003| 0.4923390885331935|
2020-06| 7584|1511371| 0.5017960513996894|
2020-05| 27556|1899522| 1.4506807502097896|
2020-04| 99671|1472249| 6.769982523336745|
2020-03|104664| 488307| 21.434056853577772|
2020-02| 1120| 18661| 6.001821981673007|
+-------+------+-------+----------------------------+

#### Attuali Positivi per Regione

In [0]:
attuali_positivi_regione = dati_covid_regioni.where(col("data") >= current_date() - expr("INTERVAL 1 days")) \
                                             .select('data','denominazione_regione', 'totale_positivi') \
                                             .withColumnRenamed('denominazione_regione', 'regione') \
                                             .withColumnRenamed('totale_positivi', 'attuali_positivi') \
                                             .withColumn('data', date_format(dati_covid_regioni.data, 'yyyy-MM-dd'))

popolazione = popolazione_regioni.groupBy('denominazione_regione') \
                                 .sum('totale_generale')

cond = [popolazione.denominazione_regione == attuali_positivi_regione.regione]
attuali_positivi_regione = attuali_positivi_regione.join(popolazione, cond, how='left') \
                                                   .drop('denominazione_regione') \
                                                   .withColumnRenamed('sum(totale_generale)', 'popolazione') \
                                                   .orderBy('regione')

attuali_positivi_regione = attuali_positivi_regione.withColumn('positivi_per_100k', (attuali_positivi_regione.attuali_positivi / attuali_positivi_regione.popolazione * 100000).cast('int')) \
                                                   .orderBy('positivi_per_100k', ascending=False)

attuali_positivi_regione.show(21)

+----------+--------------------+----------------+-----------+-----------------+
 data| regione|attuali_positivi|popolazione|positivi_per_100k|
+----------+--------------------+----------------+-----------+-----------------+
2021-04-29| Campania| 91885| 5712143| 1608|
2021-04-29| Puglia| 48429| 3953305| 1225|
2021-04-29| Basilicata| 6160| 553254| 1113|
2021-04-29| Sardegna| 17235| 1611621| 1069|
2021-04-29| Emilia-Romagna| 43651| 4464119| 977|
2021-04-29| Calabria| 14952| 1894110| 789|
2021-04-29| Lazio| 45362| 5755700| 788|
2021-04-29| Abruzzo| 8551| 1293941| 660|
2021-04-29|Friuli Venezia Gi...| 7627| 1206216| 632|
2021-04-29| Valle d'Aosta| 764| 125034| 611|
2021-04-29| Toscana| 21127| 3692555| 572|
2021-04-29| Lombardia| 52074| 10027602| 519|
2021-04-29| Sicilia| 25244| 4875290| 517|
2021-04-29| Veneto| 22255| 4879133| 456|
2021-04-29| Marche| 6182| 1512672| 408|
2021-04-29| Piemonte| 16121| 4311217| 373|
2021-04-29| Liguria| 5505| 1524826| 361|
2021-04-29| Umbria| 2955| 870165| 339|
2021-04-29| Molise| 606| 300516| 201|
2021-04-29| Trento| 1062| 545425| 194|
2021-04-29| Bolzano| 962| 532644| 180|
+----------+--------------------+----------------+-----------+-----------------+

#### Dati Critici per Mese

In [0]:
decessi_mensili = dati_covid_regioni.select(date_format('data','yyyy-MM').alias('mese'), 'denominazione_regione', 'deceduti') \
                                    .groupby('mese', 'denominazione_regione') \
                                    .max('deceduti') \
                                    .groupby('mese') \
                                    .sum('max(deceduti)') \
                                    .withColumnRenamed('sum(max(deceduti))', 'deceduti')

my_window = Window.partitionBy().orderBy("mese") 

decessi_mensili = decessi_mensili.withColumn("prev_value", lag(decessi_mensili.deceduti) \
                                 .over(my_window)) 
decessi_mensili = decessi_mensili.withColumn("diff", when(isnull(decessi_mensili.deceduti - decessi_mensili.prev_value), decessi_mensili.deceduti).otherwise(decessi_mensili.deceduti - decessi_mensili.prev_value)) \
                                 .drop('deceduti', 'prev_value') \
                                 .withColumnRenamed('diff', 'deceduti')

In [0]:
terapie_intensive_mensili = dati_covid_regioni.select('data', 'terapia_intensiva') \
                  .groupby('data') \
                  .sum('terapia_intensiva') \
                  .select(date_format('data','yyyy-MM').alias('mese'), 'sum(terapia_intensiva)') \
                  .groupby('mese') \
                  .mean('sum(terapia_intensiva)') \
                  .withColumnRenamed('avg(sum(terapia_intensiva))', 'media_terapia_intensiva')

terapie_intensive_mensili = terapie_intensive_mensili.withColumn('media_terapia_intensiva', terapie_intensive_mensili.media_terapia_intensiva.cast('int'))

In [0]:
dati_critici_mensili = decessi_mensili.join(casi_mensili, on='mese') \
                                      .join(terapie_intensive_mensili, on='mese') \
                                      .orderBy('mese', ascending=False)

dati_critici_mensili.show()

+-------+--------+------+-----------------------+
 mese|deceduti| casi|media_terapia_intensiva|
+-------+--------+------+-----------------------+
2021-04| 11198|426000| 3331|
2021-03| 11647|648200| 3127|
2021-02| 9183|372503| 2128|
2021-01| 14357|445585| 2483|
2020-12| 18583|508914| 3005|
2020-11| 16958|922206| 3231|
2020-10| 2724|364607| 785|
2020-09| 411| 45623| 194|
2020-08| 341| 21702| 58|
2020-07| 311| 7006| 57|
2020-06| 1406| 7584| 207|
2020-05| 5458| 27556| 882|
2020-04| 15539| 99671| 2975|
2020-03| 12399|104664| 1985|
2020-02| 29| 1120| 53|
+-------+--------+------+-----------------------+

### Dati Somministrazioni Vaccini

#### Percentuale Popolazione Vaccinata per Regione
Si riporta la percentuale di popolazione vaccinata (prima e seconda dose) per regione.

In [0]:
dosi_somministrate_regione = dati_vaccini_somministrazioni.groupBy('nome_area') \
                                                          .sum('prima_dose', 'seconda_dose') \
                                                          .orderBy('nome_area') \
                                                          .withColumnRenamed('nome_area', 'regione') \
                                                          .withColumnRenamed('sum(prima_dose)', 'vaccinati_prima_dose') \
                                                          .withColumnRenamed('sum(seconda_dose)', 'vaccinati_seconda_dose')

popolazione = popolazione_regioni.groupBy('denominazione_regione').sum('totale_generale')

cond = [popolazione.denominazione_regione == dosi_somministrate_regione.regione]
dosi_somministrate_regione = dosi_somministrate_regione.join(popolazione, cond, how='left') \
                                                       .drop('denominazione_regione') \
                                                       .withColumnRenamed('sum(totale_generale)', 'popolazione') \
                                                       .orderBy('regione')

dosi_somministrate_regione = dosi_somministrate_regione.withColumn('percentuale_prima_dose', dosi_somministrate_regione.vaccinati_prima_dose / dosi_somministrate_regione.popolazione * 100)
dosi_somministrate_regione = dosi_somministrate_regione.withColumn('percentuale_seconda_dose', dosi_somministrate_regione.vaccinati_seconda_dose / dosi_somministrate_regione.popolazione * 100)

dosi_somministrate_regione.show(21)

+--------------------+--------------------+----------------------+-----------+----------------------+------------------------+
 regione|vaccinati_prima_dose|vaccinati_seconda_dose|popolazione|percentuale_prima_dose|percentuale_seconda_dose|
+--------------------+--------------------+----------------------+-----------+----------------------+------------------------+
 Abruzzo| 309581| 121043| 1293941| 23.92543400355967| 9.35459963012224|
 Basilicata| 127113| 53005| 553254| 22.97552299667061| 9.580590470199944|
 Bolzano| 135678| 50849| 532644| 25.472548268637212| 9.546526385353069|
 Calabria| 368626| 157554| 1894110| 19.461699690091915| 8.318101905380363|
 Campania| 1211594| 460668| 5712143| 21.21084853793051| 8.064714066156958|
 Emilia-Romagna| 1088389| 516812| 4464119| 24.38082407749435| 11.577021132277164|
Friuli Venezia Gi...| 280141| 132166| 1206216| 23.22477897822612| 10.957075681304177|
 Lazio| 1296449| 576162| 5755700| 22.52461038622583| 10.010285456156506|
 Liguria| 408718| 182506| 1524826| 26.804238647557167| 11.968972197483515|
 Lombardia| 2329576| 928263| 10027602| 23.23163603820734| 9.257078611616217|
 Marche| 394760| 144557| 1512672| 26.096867000909647| 9.556400858877536|
 Molise| 74568| 38135| 300516| 24.813321087729108| 12.689840141623074|
 Piemonte| 1070357| 447386| 4311217| 24.82725875315485| 10.377255424628359|
 Puglia| 946911| 306417| 3953305| 23.952389203463937| 7.7509071523699795|
 Sardegna| 348692| 148441| 1611621| 21.6361042701727| 9.210664293900365|
 Sicilia| 962825| 434606| 4875290| 19.749081593094974| 8.914464575440642|
 Toscana| 858245| 371397| 3692555| 23.242578648117632| 10.057995073871615|
 Trento| 130882| 41920| 545425| 23.996333134711463| 7.685749644772426|
 Umbria| 212933| 86746| 870165| 24.470416530198296| 9.968913941608776|
 Valle d'Aosta| 32703| 12006| 125034| 26.155285762272662| 9.602188204808291|
 Veneto| 1118649| 500586| 4879133| 22.92720858398408| 10.259732620529098|
+--------------------+--------------------+----------------------+-----------+----------------------+------------------------+

#### Percentuale Popolazione Vaccinata per Fascia Anagrafica
Si riporta la percentuale di popolazione vaccinata (prima e seconda dose) per fascia anagrafica.

In [0]:
dosi_somministrate_fascia = dati_vaccini_somministrazioni.groupBy('fascia_anagrafica') \
                                                          .sum('prima_dose', 'seconda_dose') \
                                                          .orderBy('fascia_anagrafica') \
                                                          .withColumnRenamed('sum(prima_dose)', 'vaccinati_prima_dose') \
                                                          .withColumnRenamed('sum(seconda_dose)', 'vaccinati_seconda_dose')

popolazione = popolazione_regioni.groupBy('range_eta').sum('totale_generale')

cond = [popolazione.range_eta == dosi_somministrate_fascia.fascia_anagrafica]
dosi_somministrate_fascia = dosi_somministrate_fascia.join(popolazione, cond, how='left') \
                                                       .withColumnRenamed('sum(totale_generale)', 'popolazione') \
                                                       .drop('range_eta') \
                                                       .orderBy('fascia_anagrafica')

dosi_somministrate_fascia = dosi_somministrate_fascia.withColumn('percentuale_prima_dose', dosi_somministrate_fascia.vaccinati_prima_dose / dosi_somministrate_fascia.popolazione * 100)
dosi_somministrate_fascia = dosi_somministrate_fascia.withColumn('percentuale_seconda_dose', dosi_somministrate_fascia.vaccinati_seconda_dose / dosi_somministrate_fascia.popolazione * 100)

dosi_somministrate_fascia.show(9)

+-----------------+--------------------+----------------------+-----------+----------------------+------------------------+
fascia_anagrafica|vaccinati_prima_dose|vaccinati_seconda_dose|popolazione|percentuale_prima_dose|percentuale_seconda_dose|
+-----------------+--------------------+----------------------+-----------+----------------------+------------------------+
 16-19| 26780| 8814| 2298846| 1.1649323182153133| 0.38340976298542834|
 20-29| 558834| 281768| 6084382| 9.184729032463773| 4.631004430688277|
 30-39| 809536| 376001| 6854632| 11.81005778282481| 5.485356471361263|
 40-49| 1186352| 484723| 8937229| 13.274271029644646| 5.423638579698473|
 50-59| 1639836| 662665| 9414195| 17.418759649656714| 7.038998023729061|
 60-69| 2160884| 549743| 7364364| 29.342438803948312| 7.464908035507207|
 70-79| 3480887| 496476| 5968373| 58.32220941955203| 8.318447925422893|
 80-89| 3142892| 2339198| 3628160| 86.62495590051155| 64.473396983595|
 90+| 701389| 511837| 791543| 88.61034713212042| 64.66319580869265|
+-----------------+--------------------+----------------------+-----------+----------------------+------------------------+

#### Percentuale Popolazione Vaccinata per Regione e Fascia Anagrafica
Si riporta la percentuale di popolazione vaccinata (prima e seconda dose) per regione e fascia anagrafica.

In [0]:
dosi_somministrate = dati_vaccini_somministrazioni.groupBy('nome_area', 'fascia_anagrafica') \
                                                  .sum('prima_dose', 'seconda_dose') \
                                                  .orderBy('nome_area', 'fascia_anagrafica') \
                                                  .withColumnRenamed('nome_area', 'regione') \
                                                  .withColumnRenamed('sum(prima_dose)', 'vaccinati_prima_dose') \
                                                  .withColumnRenamed('sum(seconda_dose)', 'vaccinati_seconda_dose')

cond = [popolazione_regioni.denominazione_regione == dosi_somministrate.regione, popolazione_regioni.range_eta == dosi_somministrate.fascia_anagrafica]
dosi_somministrate = dosi_somministrate.join(popolazione_regioni.select('denominazione_regione', 'range_eta', 'totale_generale'), cond, how='left') \
                                       .drop('denominazione_regione', 'range_eta') \
                                       .withColumnRenamed('totale_generale', 'popolazione') \
                                       .orderBy('regione', 'fascia_anagrafica')

dosi_somministrate = dosi_somministrate.withColumn('percentuale_prima_dose', dosi_somministrate.vaccinati_prima_dose / dosi_somministrate.popolazione * 100)
dosi_somministrate = dosi_somministrate.withColumn('percentuale_seconda_dose', dosi_somministrate.vaccinati_seconda_dose / dosi_somministrate.popolazione * 100)

dosi_somministrate.show(189)

+--------------------+-----------------+--------------------+----------------------+-----------+----------------------+------------------------+
 regione|fascia_anagrafica|vaccinati_prima_dose|vaccinati_seconda_dose|popolazione|percentuale_prima_dose|percentuale_seconda_dose|
+--------------------+-----------------+--------------------+----------------------+-----------+----------------------+------------------------+
 Abruzzo| 16-19| 652| 220| 46751| 1.3946225749181835| 0.4705781694509208|
 Abruzzo| 20-29| 11806| 4888| 130083| 9.075743948094678| 3.7576009163380304|
 Abruzzo| 30-39| 18448| 8024| 150713| 12.240483568106269| 5.32402646088924|
 Abruzzo| 40-49| 28611| 9653| 190060| 15.053667262969588| 5.078922445543513|
 Abruzzo| 50-59| 42382| 12330| 204157| 20.759513511660145| 6.039469623867905|
 Abruzzo| 60-69| 59262| 11272| 167705| 35.33705017739483| 6.721326138159268|
 Abruzzo| 70-79| 64822| 6750| 130572| 49.64464050485556| 5.169561621174525|
 Abruzzo| 80-89| 68271| 55740| 84488| 80.80555818577785| 65.97386611116372|
 Abruzzo| 90+| 15327| 12166| 19515| 78.53958493466564| 62.34178836792211|
 Basilicata| 16-19| 446| 33| 22032| 2.024328249818446| 0.1497821350762527|
 Basilicata| 20-29| 4476| 1233| 61183| 7.315757645097494| 2.0152656783747123|
 Basilicata| 30-39| 7532| 2875| 65204| 11.551438562051409| 4.409238697012453|
 Basilicata| 40-49| 12579| 3937| 78467| 16.030942944167613| 5.017395847936075|
 Basilicata| 50-59| 18424| 4917| 87822| 20.97879802327435| 5.598824895811983|
 Basilicata| 60-69| 21815| 4050| 72817| 29.958663498908223| 5.561888020654517|
 Basilicata| 70-79| 23159| 1167| 51805| 44.70417913328829| 2.25267831290416|
 Basilicata| 80-89| 31066| 28151| 36107| 86.03871825407816| 77.96549145595037|
 Basilicata| 90+| 7616| 6642| 7823| 97.35395628275597| 84.90348970983|
 Bolzano| 16-19| 363| 133| 23269| 1.5600154712278138| 0.5715759164553698|
 Bolzano| 20-29| 6006| 2838| 61094| 9.830752610731004| 4.645300684191573|
 Bolzano| 30-39| 8424| 3513| 63127| 13.344527698132335| 5.564972198900628|
 Bolzano| 40-49| 12699| 5276| 75664| 16.78341086910552| 6.972932966800592|
 Bolzano| 50-59| 16853| 6717| 83539| 20.17381103436718| 8.040555908019009|
 Bolzano| 60-69| 32257| 3872| 57041| 56.550551357795264| 6.788099787871882|
 Bolzano| 70-79| 31654| 5213| 46613| 67.90809430845472| 11.18357539742132|
 Bolzano| 80-89| 22317| 18790| 27108| 82.32625055334218| 69.31533126752251|
 Bolzano| 90+| 5105| 4497| 6165| 82.80616382806164| 72.9440389294404|
 Calabria| 16-19| 747| 276| 76743| 0.973378679488683| 0.3596419217387905|
 Calabria| 20-29| 12909| 5862| 213889| 6.035373488117669| 2.7406739009486225|
 Calabria| 30-39| 22641| 11454| 236668| 9.566565822164382| 4.839691043994118|
 Calabria| 40-49| 33669| 14299| 266417| 12.637707053228587| 5.367149994182053|
 Calabria| 50-59| 49599| 20094| 286804| 17.293691859248824| 7.006178435447204|
 Calabria| 60-69| 80749| 23420| 241215| 33.47594469663993| 9.709180606512861|
 Calabria| 70-79| 76726| 15553| 175208| 43.79137938906899| 8.876877768138442|
 Calabria| 80-89| 75765| 55417| 107720| 70.33512810991459| 51.445414036390645|
 Calabria| 90+| 15821| 11179| 23058| 68.61393008933993| 48.48208864602307|
 Campania| 16-19| 3439| 1114| 261694| 1.3141302437197644| 0.42568801730265116|
 Campania| 20-29| 52366| 23934| 693479| 7.551201983044908| 3.4512941271473254|
 Campania| 30-39| 79498| 35317| 715258| 11.11459081897721| 4.937658858761454|
 Campania| 40-49| 116037| 41837| 835597| 13.886718118901815| 5.006839421395721|
 Campania| 50-59| 174023| 59317| 868684| 20.032946387869465| 6.828374875098425|
 Campania| 60-69| 267698| 66755| 670867| 39.90328932560403| 9.950556518654219|
 Campania| 70-79| 286151| 50031| 484380| 59.075725669928566| 10.328874024526199|
 Campania| 80-89| 197244| 158802| 255273| 77.26786616680965| 62.208694221480535|
 Campania| 90+| 35138| 23561| 49044| 71.64586901557784| 48.04053502976919|
 Emilia-Romagna| 16-19| 2269| 726| 160045| 1.4177262644881128| 0.4536224186947421|
 Emilia-Romagna| 20-29| 45013| 24238

#### Categorie

In [0]:
dosi_somministrate_categorie_mensili = dati_vaccini_somministrazioni.select(date_format('data_somministrazione','yyyy-MM').alias('mese'), \
                                                                            'categoria_operatori_sanitari_sociosanitari', 'categoria_personale_non_sanitario', 'categoria_ospiti_rsa', \
                                                                            'categoria_60_69', 'categoria_70_79', 'categoria_over80', 'categoria_forze_armate', 'categoria_personale_scolastico', \
                                                                            'categoria_soggetti_fragili', 'categoria_altro') \
                                                                    .groupby('mese') \
                                                                    .agg(sum('categoria_operatori_sanitari_sociosanitari').alias('operatori_sanitari_sociosanitari'), \
                                                                         sum('categoria_personale_non_sanitario').alias('personale_non_sanitario'), \
                                                                         sum('categoria_ospiti_rsa').alias('ospiti_rsa'), \
                                                                         sum('categoria_60_69').alias('60_69'), \
                                                                         sum('categoria_70_79').alias('70_79'), \
                                                                         sum('categoria_over80').alias('over80'), \
                                                                         sum('categoria_forze_armate').alias('forze_armate'), \
                                                                         sum('categoria_personale_scolastico').alias('personale_scolastico'), \
                                                                         sum('categoria_soggetti_fragili').alias('soggetti_fragili'), \
                                                                         sum('categoria_altro').alias('altro')) \
                                                                      .orderBy('mese')

dosi_somministrate_categorie_mensili.show()

+-------+--------------------------------+-----------------------+----------+------+-------+-------+------------+--------------------+----------------+------+
 mese|operatori_sanitari_sociosanitari|personale_non_sanitario|ospiti_rsa| 60_69| 70_79| over80|forze_armate|personale_scolastico|soggetti_fragili| altro|
+-------+--------------------------------+-----------------------+----------+------+-------+-------+------------+--------------------+----------------+------+
2020-12| 33953| 1493| 3262| 163| 42| 51| 0| 8| 1| 843|
2021-01| 1458485| 229708| 200884| 11933| 4473| 17782| 177| 322| 139| 55294|
2021-02| 901157| 269826| 212539| 14415| 8363| 639844| 67404| 176824| 2616| 76154|
2021-03| 598487| 280755| 154476| 86808| 400656|2823196| 215144| 847361| 438944|116001|
2021-04| 235059| 103871| 80245|862183|2447885|2591901| 39302| 134432| 2468714|105040|
+-------+--------------------------------+-----------------------+----------+------+-------+-------+------------+--------------------+----------------+------+

In [0]:
dosi_somministrate_mensili = dati_vaccini_somministrazioni.select(date_format('data_somministrazione','yyyy-MM').alias('mese'), 'prima_dose', 'seconda_dose') \
                                                          .groupby('mese') \
                                                          .sum('prima_dose', 'seconda_dose')

dosi_somministrate_mensili = dosi_somministrate_mensili.withColumnRenamed('sum(prima_dose)', 'vaccinati_prima_dose') \
                                                       .withColumnRenamed('sum(seconda_dose)', 'vaccinati_seconda_dose')


totale_dosi_somministrate_mensili = dosi_somministrate_mensili.withColumn('totale_dosi', dosi_somministrate_mensili.vaccinati_prima_dose + dosi_somministrate_mensili.vaccinati_seconda_dose)

percentuale_dosi_categorie = totale_dosi_somministrate_mensili.join(dosi_somministrate_categorie_mensili, on='mese') \
                                                              .drop('vaccinati_prima_dose', 'vaccinati_seconda_dose') \
                                                              .withColumn('operatori_sanitari_sociosanitari', round(dosi_somministrate_categorie_mensili.operatori_sanitari_sociosanitari / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('personale_non_sanitario', round(dosi_somministrate_categorie_mensili.personale_non_sanitario / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('ospiti_rsa', round(dosi_somministrate_categorie_mensili.ospiti_rsa / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('60_69', round(dosi_somministrate_categorie_mensili['60_69'] / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('70_79', round(dosi_somministrate_categorie_mensili['70_79'] / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('over80', round(dosi_somministrate_categorie_mensili.over80 / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('forze_armate', round(dosi_somministrate_categorie_mensili.forze_armate / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('personale_scolastico', round(dosi_somministrate_categorie_mensili.personale_scolastico / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('soggetti_fragili', round(dosi_somministrate_categorie_mensili.soggetti_fragili / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('altro', round(dosi_somministrate_categorie_mensili.altro / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .drop('totale_dosi') \
                                                              .orderBy('mese')
    
percentuale_dosi_categorie.show()

+-------+--------------------------------+-----------------------+----------+-----+------+------+------------+--------------------+----------------+-----+
 mese|operatori_sanitari_sociosanitari|personale_non_sanitario|ospiti_rsa|60_69| 70_79|over80|forze_armate|personale_scolastico|soggetti_fragili|altro|
+-------+--------------------------------+-----------------------+----------+-----+------+------+------------+--------------------+----------------+-----+
2020-12| 85.275| 3.75| 8.193|0.409| 0.105| 0.128| 0.0| 0.02| 0.003|2.117|
2021-01| 73.691| 11.606| 10.15|0.603| 0.226| 0.898| 0.009| 0.016| 0.007|2.794|
2021-02| 38.037| 11.389| 8.971|0.608| 0.353|27.007| 2.845| 7.464| 0.11|3.214|
2021-03| 10.039| 4.709| 2.591|1.456| 6.72|47.355| 3.609| 14.213| 7.363|1.946|
2021-04| 2.592| 1.145| 0.885|9.507|26.993|28.581| 0.433| 1.482| 27.223|1.158|
+-------+--------------------------------+-----------------------+----------+-----+------+------+------------+--------------------+----------------+-----+

#### Fornitori e Fasce Anagrafiche

In [0]:
dosi_fascia_fornitore = dati_vaccini_somministrazioni.groupBy('fascia_anagrafica', 'fornitore') \
                                                          .sum('prima_dose', 'seconda_dose') \
                                                          .orderBy('fascia_anagrafica', 'fornitore') 

totale_dosi_somministrate_fascia = dosi_somministrate_fascia.select('fascia_anagrafica', 'vaccinati_prima_dose', 'vaccinati_seconda_dose') \
                                                            .withColumn('totale_dosi', dosi_somministrate_fascia.vaccinati_prima_dose + dosi_somministrate_fascia.vaccinati_seconda_dose) \
                                                            .drop('vaccinati_prima_dose', 'vaccinati_seconda_dose')

dosi_fascia_fornitore = dosi_fascia_fornitore.join(totale_dosi_somministrate_fascia, on='fascia_anagrafica')

dosi_fascia_fornitore = dosi_fascia_fornitore.withColumn('percentuale_dosi', round((dosi_fascia_fornitore['sum(prima_dose)'] + dosi_fascia_fornitore['sum(seconda_dose)']) *100 /  dosi_fascia_fornitore.totale_dosi,3)) \
                                             .drop('sum(prima_dose)', 'sum(seconda_dose)', 'totale_dosi') \
                                             .orderBy('fascia_anagrafica', 'fornitore')

dosi_fascia_fornitore.show(100)

+-----------------+--------------------+----------------+
fascia_anagrafica| fornitore|percentuale_dosi|
+-----------------+--------------------+----------------+
 16-19| Janssen| 0.022|
 16-19| Moderna| 7.473|
 16-19| Pfizer/BioNTech| 85.455|
 16-19|Vaxzevria (AstraZ...| 7.049|
 20-29| Janssen| 0.105|
 20-29| Moderna| 4.439|
 20-29| Pfizer/BioNTech| 74.612|
 20-29|Vaxzevria (AstraZ...| 20.844|
 30-39| Janssen| 0.114|
 30-39| Moderna| 4.408|
 30-39| Pfizer/BioNTech| 70.139|
 30-39|Vaxzevria (AstraZ...| 25.339|
 40-49| Janssen| 0.089|
 40-49| Moderna| 4.735|
 40-49| Pfizer/BioNTech| 66.803|
 40-49|Vaxzevria (AstraZ...| 28.373|
 50-59| Janssen| 0.062|
 50-59| Moderna| 5.192|
 50-59| Pfizer/BioNTech| 69.177|
 50-59|Vaxzevria (AstraZ...| 25.569|
 60-69| Janssen| 0.541|
 60-69| Moderna| 6.35|
 60-69| Pfizer/BioNTech| 61.753|
 60-69|Vaxzevria (AstraZ...| 31.355|
 70-79| Janssen| 0.633|
 70-79| Moderna| 7.092|
 70-79| Pfizer/BioNTech| 48.443|
 70-79|Vaxzevria (AstraZ...| 43.832|
 80-89| Janssen| 0.009|
 80-89| Moderna| 9.481|
 80-89| Pfizer/BioNTech| 89.36|
 80-89|Vaxzevria (AstraZ...| 1.151|
 90+| Janssen| 0.006|
 90+| Moderna| 13.671|
 90+| Pfizer/BioNTech| 85.639|
 90+|Vaxzevria (AstraZ...| 0.684|
+-----------------+--------------------+----------------+

### Correlazione Dati

#### Dosi Somministrate e Media terapie intensive

In [0]:
dosi_somministrate_terapie_intensive = dosi_somministrate_mensili.join(terapie_intensive_mensili, on='mese', how='left') \
                                                                 .orderBy('mese', ascending=False)


dosi_somministrate_terapie_intensive.show()

+-------+--------------------+----------------------+-----------------------+
 mese|vaccinati_prima_dose|vaccinati_seconda_dose|media_terapia_intensiva|
+-------+--------------------+----------------------+-----------------------+
2021-04| 6593619| 2475013| 3331|
2021-03| 4146802| 1815026| 3127|
2021-02| 1588266| 780876| 2128|
2021-01| 1338887| 640310| 2483|
2020-12| 39816| 0| 3005|
+-------+--------------------+----------------------+-----------------------+

#### Dosi Somministrate e Media terapie intensive

In [0]:
attuali_terapie_intensive_regioni = dati_covid_regioni.where(col("data") >= current_date() - expr("INTERVAL 1 days")) \
                                             .select('data', 'denominazione_regione', 'terapia_intensiva') \
                                             .withColumnRenamed('denominazione_regione', 'regione') \
                                             .withColumn('data', date_format(dati_covid_regioni.data, 'yyyy-MM-dd'))

In [0]:
attuali_dosi_somministrate_terapie_intensive = attuali_terapie_intensive_regioni.join(dosi_somministrate_regione, on='regione') \
                                                                                .select('data', 'regione', 'percentuale_prima_dose', 'percentuale_seconda_dose', 'terapia_intensiva') \
                                                                                .orderBy('regione')

attuali_dosi_somministrate_terapie_intensive.show(21)

+----------+--------------------+----------------------+------------------------+-----------------+
 data| regione|percentuale_prima_dose|percentuale_seconda_dose|terapia_intensiva|
+----------+--------------------+----------------------+------------------------+-----------------+
2021-04-29| Abruzzo| 23.92543400355967| 9.35459963012224| 39|
2021-04-29| Basilicata| 22.97552299667061| 9.580590470199944| 6|
2021-04-29| Bolzano| 25.472548268637212| 9.546526385353069| 3|
2021-04-29| Calabria| 19.461699690091915| 8.318101905380363| 48|
2021-04-29| Campania| 21.21084853793051| 8.064714066156958| 137|
2021-04-29| Emilia-Romagna| 24.38082407749435| 11.577021132277164| 234|
2021-04-29|Friuli Venezia Gi...| 23.22477897822612| 10.957075681304177| 30|
2021-04-29| Lazio| 22.52461038622583| 10.010285456156506| 291|
2021-04-29| Liguria| 26.804238647557167| 11.968972197483515| 67|
2021-04-29| Lombardia| 23.23163603820734| 9.257078611616217| 557|
2021-04-29| Marche| 26.096867000909647| 9.556400858877536| 70|
2021-04-29| Molise| 24.813321087729108| 12.689840141623074| 9|
2021-04-29| Piemonte| 24.82725875315485| 10.377255424628359| 220|
2021-04-29| Puglia| 23.952389203463937| 7.7509071523699795| 230|
2021-04-29| Sardegna| 21.6361042701727| 9.210664293900365| 48|
2021-04-29| Sicilia| 19.749081593094974| 8.914464575440642| 168|
2021-04-29| Toscana| 23.242578648117632| 10.057995073871615| 254|
2021-04-29| Trento| 23.996333134711463| 7.685749644772426| 20|
2021-04-29| Umbria| 24.470416530198296| 9.968913941608776| 33|
2021-04-29| Valle d'Aosta| 26.155285762272662| 9.602188204808291| 7|
2021-04-29| Veneto| 22.92720858398408| 10.259732620529098| 169|
+----------+--------------------+----------------------+------------------------+-----------------+

# IDEE
* Dubito che metteremo mano, ma ci sono somministrazioni di vaccini a singola dose.